In [29]:
library(readr)
library(dplyr)
library(tidyr)
library(stringr)
library(doMC)
library(lfe)
library(lubridate)
library(ggplot2)
library(ggsci)
library(xgboost)
registerDoMC(24)

In [30]:
fp  <- '/pool001/mfzhao/'
fb  <- read_tsv(str_c(fp, 'mobility/fb/movement-range-2020-08-08.txt'))
sg  <- read_rds(str_c(fp, 'PROCESSED_DATA/panel_pre_xgr.RDS'))
sci <- read_csv(str_c(fp, 'PROCESSED_DATA/processed_sci.csv'))

Parsed with column specification:
cols(
  ds = col_date(format = ""),
  country = col_character(),
  polygon_source = col_character(),
  polygon_id = col_character(),
  polygon_name = col_character(),
  all_day_bing_tiles_visited_relative_change = col_double(),
  all_day_ratio_single_tile_users = col_double(),
  baseline_name = col_character(),
  baseline_type = col_character()
)

Parsed with column specification:
cols(
  user_county = col_character(),
  fr_county = col_character(),
  scaled_sci = col_double(),
  n = col_double()
)



In [31]:
fb %>%
    filter(country == 'USA') %>%
    select(key = polygon_id, 
           date = ds, 
           all_day_bing_tiles_visited_relative_change,
           all_day_ratio_single_tile_users) -> fb

In [32]:
sg %>%
    select(key) %>%
    distinct() %>%
    inner_join(fb %>%
               group_by(key) %>%
               tally() %>%
               filter(n == max(n)) %>%
               select(key)) -> keys

Joining, by = "key"



In [33]:
sg %>% 
    filter(date >= as.Date('2020-03-01')) %>%
    inner_join(keys) %>%
    select(!matches('alter')) %>%
    arrange(date, key) -> sg

Joining, by = "key"



In [34]:
fb %>%
    inner_join(keys) %>%
    filter(date < as.Date('2020-07-01')) %>%
    arrange(date, key) %>%
    rename(btvrc = all_day_bing_tiles_visited_relative_change,
           rstu  = all_day_ratio_single_tile_users) %>%
    mutate(rnstu     = 1 - rstu,
           log_rnstu = log(rnstu)) %>%
    select(-rstu) -> fb

Joining, by = "key"



In [35]:
inner_join(sg, fb) -> df

Joining, by = c("key", "date")



In [36]:
sci %>%
    inner_join(keys, by = c('fr_county' = 'key')) %>%
    inner_join(keys, by = c('user_county' = 'key')) -> sci

In [37]:
sci %>%
    mutate(user_state = str_sub(user_county, 1, 2),
           fr_state   = str_sub(fr_county, 1, 2)) %>%
    group_by(user_county) %>%
    mutate(w = ifelse(user_state == fr_state, 0, scaled_sci * n),
           w = w/sum(w)) %>%
    select(key = user_county, fr_county, w) %>%
    spread(key = fr_county, value = w) %>%
    ungroup(key) %>%
    arrange(key) %>%
    select(-key) %>%
    as.matrix() -> stateWM

rownames(stateWM) <- keys$key

In [38]:
weightedAlters <- function(df, wm, ...) {
    df %>% 
        select(date, key, ...) %>%
        spread(key = key, value = ...) %>%
        ungroup() %>%
        select(-date) %>%
        as.matrix() -> txn_data
    
    df %>%
        ungroup() %>%
        select(date) %>%
        distinct() %>%
        arrange(date) -> dates
    
    outMatrix <- tcrossprod(txn_data, wm)
    colnames(outMatrix) <- colnames(txn_data)
    
    data.frame(dates, outMatrix) %>%
        gather(key = 'key', value = 'value', -date) %>%
        arrange(date, key) %>%
        select(-date, -key) -> out_df
    return(out_df$value)
}

In [39]:
df %>%
    select(p1sdp, p2shp, p3rop) %>%
    colnames() -> cols_to_alterize
cols_to_alterize

[1] "p1sdp" "p2shp" "p3rop"

In [40]:
stalters <- foreach(i = 1:length(cols_to_alterize), .combine = cbind) %dopar% 
    weightedAlters(df, stateWM, cols_to_alterize[i])
colnames(stalters) <- str_c('stalter_', cols_to_alterize)

In [41]:
df %>%
    bind_cols(as.data.frame(stalters)) -> df

In [42]:
set.seed(2345)
df %>%
    ungroup() %>%
    select(key) %>%
    distinct() %>% 
    mutate(i = sample(1:n(), n(), replace = F),
           fold = i %% 3 + 1) %>%
    select(-i) -> folds

df %>%
    inner_join(folds) %>%
    arrange(date, key) %>%
    ungroup() -> df

df %>%
    ungroup() %>%
    mutate(PRCP.r_fe = felm(PRCP ~ 0 | key + date, ., weights = df$n)$resid,
           TMAX.r_fe = felm(TMAX ~ 0 | key + date, ., weights = df$n)$resid) %>%
    select(date, key, PRCP.r_fe, TMAX.r_fe, fold, n) -> rdf

folds <- list(which(folds$fold %in% 1), 
              which(folds$fold %in% 2), 
              which(folds$fold %in% 3))

Joining, by = "key"



In [43]:
XGresidualizer <- function(Y, colname) {
    print(colname)
    rdf %>%
        ungroup() %>% 
        mutate(Y = Y,
               Y.r = felm(Y ~ 0 | key + date, ., weights = rdf$n)$resid) -> temp_df
  
    folds <- list(which(temp_df$fold %in% 1), 
                  which(temp_df$fold %in% 2), 
                  which(temp_df$fold %in% 3))

    dm    <- xgb.DMatrix(data = model.matrix(~ 0 + PRCP.r_fe + TMAX.r_fe, temp_df), label = temp_df$Y.r)
    param <- list(max_depth=2, eta=.5, silent=1, objective='reg:linear')
    fit   <- xgb.cv(params = param, 
                    data = dm, 
                    folds = folds,
                    nrounds = 100, 
                    early_stopping_rounds = 3, 
                    weight = temp_df$n)
    best_n <- fit$best_iteration
    for (i in 1:3) {
        tr  <- temp_df %>% filter(fold != i)
        trm <- xgb.DMatrix(data = model.matrix(~ 0 + PRCP.r_fe + TMAX.r_fe, tr), label = tr$Y.r)
        fit <- xgb.train(params = param, 
                         data = trm, 
                         nrounds = best_n, 
                         weight = tr$n)
        te  <- temp_df %>% filter(fold == i)
        tem <- xgb.DMatrix(data = model.matrix(~ 0 + PRCP.r_fe + TMAX.r_fe, te), label = te$Y.r)
        te %>%
            select(date, key) %>%
            mutate(pred = predict(fit, newdata = tem)) -> pred_df
        assign(str_c('temp',i), pred_df) %>%
        select(-pred, -date, -key)
    }
    out <- bind_rows(temp1, temp2, temp3) %>%
        arrange(date, key) %>%
        mutate(tempname = temp_df$Y.r - pred) %>%
        select(-pred, -date, -key)
    
    colnames(out) <- str_c(colname, '.r')
    return(out)
}

In [44]:
cols_to_xgr <- colnames(
    df %>%
        select(
            log_mcbgv, 
            log_pgt2kmt, 
            log_pgt1hafh, 
            log_pnchd, 
            btvrc, 
            log_rnstu,
            p1sdp,
            p2shp,
            p3rop,
            stalter_p1sdp,
            stalter_p2shp,
            stalter_p3rop
        )
)

foreach(i = 1:length(cols_to_xgr), .combine = cbind) %do% 
    XGresidualizer(df[[cols_to_xgr[i]]], cols_to_xgr[i]) -> xg.residuals

[1] "log_mcbgv"
[1]	train-rmse:0.254527+0.000004	test-rmse:0.254533+0.000011 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 3 rounds.

[2]	train-rmse:0.133700+0.000010	test-rmse:0.133710+0.000025 
[3]	train-rmse:0.078341+0.000016	test-rmse:0.078357+0.000040 
[4]	train-rmse:0.056540+0.000019	test-rmse:0.056563+0.000057 
[5]	train-rmse:0.049583+0.000020	test-rmse:0.049612+0.000068 
[6]	train-rmse:0.047666+0.000020	test-rmse:0.047699+0.000074 
[7]	train-rmse:0.047156+0.000021	test-rmse:0.047194+0.000077 
[8]	train-rmse:0.047013+0.000019	test-rmse:0.047051+0.000079 
[9]	train-rmse:0.046973+0.000019	test-rmse:0.047011+0.000080 
[10]	train-rmse:0.046954+0.000022	test-rmse:0.046994+0.000083 
[11]	train-rmse:0.046944+0.000025	test-rmse:0.046985+0.000079 
[12]	train-rmse:0.046940+0.000025	test-rmse:0.046981+0.000078 
[13]	train-rmse:0.046936+0.000025	test-rmse:0.046979+0.000079 
[14]	train-rmse:0.046930+0.000025	test-rmse

In [45]:
ldvs <- c('log_mcbgv.r', 'log_pgt2kmt.r', 'log_pgt1hafh.r', 'log_pnchd.r', 'btvrc.r', 'log_rnstu.r')
f1 <- ' p1sdp.r + p2shp.r + p3rop.r '  
f2 <- ' p1sdp.r + p2shp.r + p3rop.r + stalter_p1sdp.r + stalter_p2shp.r + stalter_p3rop.r '

In [46]:
df %>%
    select(date, key, n, cluster) %>%
    bind_cols(xg.residuals) -> df.r

In [47]:
get_coefs <- function(dv, vars, ...) {
    lhs <- str_c(dv, ' ~ ')
    rhs <- str_c(vars, '| key + date | 0 | cluster')
    form <- as.formula(str_c(lhs, rhs))
    model <- felm(form, df.r, weights = df.r$n, ...)
    
    as.data.frame(summary(model)$coef) %>%
        mutate(var   = rownames(.),
               dv    = dv,
               model = ifelse(vars == f1, 'base', 'ap')) %>%
        filter(!str_detect(var, '[pPtT][rRmM][cCaA][pPxX]')) %>%
        select(7, 6, 5, 1, 2, 3, 4) -> out
    
    colnames(out) <- c('model', 'dv', 'var', 'estimate', 'se', 't', 'p-val')
    return(out)
}

In [48]:
foreach(dv = ldvs, .combine = rbind) %:% 
    foreach(f = c(f1, f2), .combine = rbind) %dopar%
    get_coefs(dv, f) -> coefs

In [28]:
options(repr.plot.width=15, repr.plot.height=10)
pn <- c('p1sdp' = "Initial Policies",
        'p2shp' = "Shelter-in-place",
        'p3rop' = "Reopening",
        'a'     = 'ego state policy',
        'b'     = 'alter state policy')

coefs %>%
    mutate(dv   = str_sub(dv, 1, -3),
           var  = str_sub(var, 1, -3),
           type = ifelse(dv == 'log_rnstu' | dv == 'btvrc', 'fb', 'sg'), 
           type = factor(type, levels = c('sg', 'fb'))) %>%
    mutate(vtype = ifelse(str_detect(var, 'stalter'), 'b', 'a'),
           var   = str_replace(var, 'stalter_', ''),
           model = factor(model, levels = c('base', 'ap')),
           dv = factor(dv, levels = c('log_mcbgv', 'log_pgt2kmt', 'log_pgt1hafh', 'log_pnchd', 'btvrc', 'log_rnstu'))) %>%
    ggplot(aes(x = model, y = estimate, color = dv, shape = type)) +
    geom_hline(aes(yintercept = 0), linetype = 2) +
    geom_point(position = position_dodge(width = .5), size = 2) + 
    geom_linerange(aes(ymin = estimate - 1.98 * se, ymax = estimate + 1.98 * se), 
                   position = position_dodge(width = .5), size = .5) +
    facet_grid(vtype~var, scales = 'free_y', labeller = as_labeller(pn)) +
    xlab('') + 
    ylab('') +
    scale_shape(guide = 'none') + 
    scale_color_d3() +
    labs(color = "Outcome", shape = '') +
    theme_light() +
    theme(text = element_text(size=20),
          legend.position = 'bottom') -> p

ggsave('/home/mfzhao/SI_plots/rc1_fb.pdf', p, device = 'pdf', width = 6.5, height = 6.5, scale = 2)

In [52]:
df %>%
    select(date) %>%
    distinct()

date
<date>
2020-03-01
2020-03-02
2020-03-03
2020-03-04
2020-03-05
2020-03-06
2020-03-07
2020-03-08
2020-03-09
